In [1]:
%reload_ext autoreload
%autoreload 2

<center><h1>Graph Convolution Network using PyTorch Geometric</h1></center>

$$\mathbf{x}_i^{(k)} = \gamma^{(k)} \left( \mathbf{x}_i^{(k-1)}, \square_{j \in \mathcal{N}(i)} \, \phi^{(k)}\left(\mathbf{x}_i^{(k-1)}, \mathbf{x}_j^{(k-1)},\mathbf{e}_{j,i}\right) \right),$$

<h3><a href="https://arxiv.org/abs/1609.02907">GCN layer from Kipf and Welling</a><h3>

<div style="background-color:rgba(192,192,192,0.3); padding:10px 0; font-size:110%; color:black;">
The Kipf and Welling paper describes the Graph Convolution operation as:
</div>

$$\mathbf{x}_i^{(k)} = \sum_{j \in \mathcal{N}(i) \cup \{ i \}} \frac{1}{\sqrt{\deg(i)} \cdot \sqrt{\deg(j)}} \cdot \left( \mathbf{\Theta} \cdot \mathbf{x}_j^{(k-1)} \right)$$<br>$\mathbf{\Theta}\text{ is the weight matrix by which the neighboring nodes are transformed.}$<br>$\mathcal{N}(i)\text{ are the Neighborhood nodes of node }\mathcal{i}.$<br>$\mathbf{x}_j^{(k-1)}\text{ is the feature of node }\mathcal{j}\text{ at layer }\mathcal{(k-1)}.$<br>$\mathbf{deg(i)}\text{ gives the degree of node }\mathcal{i}\text{.}$

<div style="background-color:rgba(192,192,192,0.3); padding:10px 0; font-size:110%; color:black">
The operation can be broken down into the following steps:
<ol>
    <li> Add self-loops to the Adjacency matrix.
    <li> Compute normalization coefficients.
    <li> Linearly transform node feature matrix.
    <li> Normalize node features in ϕ.
    <li> Sum up neighboring node features (if "add" aggregation).
</ol>
</div>

<h3>A Graph network</h3>

In [2]:
import torch
from torch_geometric.data import Data

In [3]:
#define property of the nodes or node features
x = torch.tensor([[2,1], [5,6], [3,7], [12,0]], dtype=torch.float)

In [4]:
#define edges between the nodes
edge_index = torch.tensor([[0, 1, 2, 0, 3],
                           [1, 0, 1, 3, 2]], dtype=torch.long)

In [5]:
#define the property of edges
edge_attr = torch.tensor([4,10,3,1,5],dtype=torch.float)

In [6]:
#define the classes the nodes belong to
y = torch.tensor([0, 1, 0, 1], dtype=torch.float)

<div style="background-color:rgba(192,192,192,0.3); padding:1px 0; font-size:105%; color:black">
<p>With these defined, the PyTorch Geometric's <b>Data</b> object now can be initialized</p>
</div>

In [7]:
g = Data(x=x,edge_index=edge_index,edge_attr=edge_attr,y=y)

<center><img src="tmp/viz_objects/1.png" alt="Input" style="width: 500px;"/></center>

<div style="background-color:rgba(192,192,192,0.3); font-size:105%; color:black">
<h4> Step 1:</h4>
<p>Add self-loops to the Adjacency matrix.</p>
</div>

<div style="background-color:rgba(192,192,192,0.3); font-size:105%; color:black">
<p>This step is required to aggregate the node's own feature with the features of its neighboring nodes:</p>
</div>

<div style="background-color:rgba(192,192,192,0.3); font-size:105%; color:black">
<p>Before:</p>
</div>

In [8]:
g.edge_index

tensor([[0, 1, 2, 0, 3],
        [1, 0, 1, 3, 2]])

In [9]:
g.edge_attr

tensor([ 4., 10.,  3.,  1.,  5.])

<div style="background-color:rgba(192,192,192,0.3); font-size:105%; color:black">
<p>After:</p>
</div>

In [10]:
from torch_geometric.utils import add_remaining_self_loops

In [11]:
new_edge_index, new_edge_attr = add_remaining_self_loops(
            edge_index, edge_attr, fill_value=1., num_nodes=4)

In [12]:
new_edge_index

tensor([[0, 1, 2, 0, 3, 0, 1, 2, 3],
        [1, 0, 1, 3, 2, 0, 1, 2, 3]])

In [13]:
new_edge_attr

tensor([ 4., 10.,  3.,  1.,  5.,  1.,  1.,  1.,  1.])

<center><img src="tmp/viz_objects/2.png" alt="Add-Self-Loop" style="width: 500px;"/></center>

<div style="background-color:rgba(192,192,192,0.3); font-size:105%; color:black">
<h4> Step 2:</h4>
<p>Compute normalization coefficients.</p>
</div>

In [14]:
from torch_scatter import scatter_add

In [15]:
#compute the degree of each node (Sum of weights in edges incoming towards a node)
row, col = new_edge_index[0], new_edge_index[1]
deg = scatter_add(new_edge_attr, col, dim=0, dim_size=4)#dim_size = num_nodes
#Compute the normalization coefficients
deg_inv_sqrt = deg.pow_(-0.5)
deg_inv_sqrt.masked_fill_(deg_inv_sqrt == float('inf'), 0)
normalized_coefficients = deg_inv_sqrt[row] * new_edge_attr * deg_inv_sqrt[col]

In [16]:
normalized_coefficients

tensor([0.4264, 1.0660, 0.4330, 0.2132, 1.4434, 0.0909, 0.1250, 0.1667, 0.5000])

<div style="background-color:rgba(192,192,192,0.3); font-size:105%; color:black">
<p>Normalizing:</p>
</div>

<center><img src="tmp/viz_objects/3.png" alt="Normalize" style="width: 700px;"/></center>

<div style="background-color:rgba(192,192,192,0.3); font-size:105%; color:black">
<p>After normalizing:</p>
</div>`

<center><img src="tmp/viz_objects/3_simplify.png" alt="Simplify" style="width: 500px;"/></center>

<div style="background-color:rgba(192,192,192,0.3); font-size:105%; color:black">
<h4> Step 3:</h4>
<p>Linearly transform node feature matrix.</p>
</div>

In [47]:
from torch.nn import Parameter
from torch_geometric.nn.inits import glorot, zeros

In [130]:
in_channels = 2
out_channels = 2 # for vizualization purpose, we are setting it to 2

In [131]:
torch.manual_seed(42)
weight = Parameter(torch.Tensor(in_channels, out_channels))
bias = Parameter(torch.Tensor(out_channels))
glorot(weight)
zeros(bias)

In [143]:
x = torch.matmul(g.x, weight)

In [144]:
x

tensor([[ 1.5858,  3.1582],
        [ 2.9603, 11.8331],
        [ 0.8006, 10.9251],
        [11.2364, 12.1986]], grad_fn=<MmBackward>)

<center><img src="tmp/viz_objects/4_new.png" alt="Convolve" style="width: 500px;"/></center>

<div style="background-color:rgba(192,192,192,0.3); font-size:105%; color:black">
<h4> Step 4:</h4>
<p>Normalize node features in ϕ.</p>
</div>

<div style="background-color:rgba(192,192,192,0.3); font-size:105%; color:black">
<p>Preparing the linearly transformed node features.</p>
</div>

In [147]:
x = x.index_select(-2, new_edge_index[0])

In [148]:
x

tensor([[ 1.5858,  3.1582],
        [ 2.9603, 11.8331],
        [ 0.8006, 10.9251],
        [ 1.5858,  3.1582],
        [11.2364, 12.1986],
        [ 1.5858,  3.1582],
        [ 2.9603, 11.8331],
        [ 0.8006, 10.9251],
        [11.2364, 12.1986]], grad_fn=<IndexSelectBackward>)

<div style="background-color:rgba(192,192,192,0.3); font-size:105%; color:black">
<p>Preparing the normalization coefficients.</p>
</div>

In [150]:
normalized_coefficients = normalized_coefficients.view(-1, 1)

In [151]:
normalized_coefficients

tensor([[0.4264],
        [1.0660],
        [0.4330],
        [0.2132],
        [1.4434],
        [0.0909],
        [0.1250],
        [0.1667],
        [0.5000]])

In [153]:
normalized_node_features = normalized_coefficients * x

In [154]:
normalized_node_features

tensor([[ 0.6762,  1.3466],
        [ 3.1557, 12.6142],
        [ 0.3467,  4.7307],
        [ 0.3381,  0.6733],
        [16.2183, 17.6071],
        [ 0.1442,  0.2871],
        [ 0.3700,  1.4791],
        [ 0.1334,  1.8208],
        [ 5.6182,  6.0993]], grad_fn=<MulBackward0>)

$$\begin{pmatrix}
     1.5858 &  3.1582 \\
     2.9603 & 11.8331 \\
     0.8006 & 10.9251 \\
     1.5858 &  3.1582 \\
    11.2364 & 12.1986 \\
     1.5858 &  3.1582 \\
     2.9603 & 11.8331 \\
     0.8006 & 10.9251 \\
    11.2364 & 12.1986 \\
    \end{pmatrix}
    \bullet
 \begin{pmatrix}
 0.4264\\
 1.0660\\
 0.4330\\
 0.2132\\
 1.4434\\
 0.0909\\
 0.1250\\
 0.1667\\
 0.5000\\
\end{pmatrix}
=
\begin{pmatrix}
0.6762 &  1.3466\\
3.1557 & 12.6142\\
0.3467 &  4.7307\\
0.3381 &  0.6733\\
16.2183 & 17.6071\\
0.1442 &  0.2871\\
0.3700 &  1.4791\\
0.1334 &  1.8208\\
5.6182 &  6.0993\\
\end{pmatrix}$$

In [159]:
new_edge_index

tensor([[0, 1, 2, 0, 3, 0, 1, 2, 3],
        [1, 0, 1, 3, 2, 0, 1, 2, 3]])

<div style="background-color:rgba(192,192,192,0.3); font-size:105%; color:black">
<h4> Step 5:</h4>
<p>Sum up neighboring node features (if <b>"add"</b> aggregation).</p>
</div>

<div style="background-color:rgba(192,192,192,0.3); font-size:105%; color:black">
<p>Other techniques can also be used for aggregation:
<ul>
<li> Summation
<li> Average
<li> Max
</ul>
</p>
</div>

In [157]:
from torch_scatter import scatter

In [164]:
new_edge_index

tensor([[0, 1, 2, 0, 3, 0, 1, 2, 3],
        [1, 0, 1, 3, 2, 0, 1, 2, 3]])

In [173]:
new_x = scatter(normalized_node_features, index=new_edge_index[1], dim=-2, dim_size=4,
                           reduce="add")

In [174]:
new_x

tensor([[ 3.2998, 12.9013],
        [ 1.3929,  7.5565],
        [16.3517, 19.4280],
        [ 5.9563,  6.7726]], grad_fn=<ScatterAddBackward>)

<center><img src="tmp/viz_objects/5.png" alt="Updated" style="width: 500px;"/></center>